In [1]:
pip install pandas numpy matplotlib seaborn scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Load data - use read_excel for .xlsx files
df = pd.read_excel('online_retail.xlsx', sheet_name='Online Retail')   # ← this is the key change


# Preview
print("Dataset shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())
print("\nInfo:")
print(df.info())
print("\nBasic stats:")
print(df.describe())

print("\nData loaded successfully!")

Dataset shape: (541909, 8)

First 5 rows:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Cou

In [4]:

print("Original shape:", df.shape)
print("Missing values per column:\n", df.isnull().sum())

# 1. Drop rows with missing CustomerID (important for customer-level analysis)
df = df.dropna(subset=['CustomerID'])

# 2. Convert InvoiceDate to datetime (if not already)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# 3. Remove cancelled/negative quantity transactions
# (Invoices starting with 'C' or negative Quantity usually mean returns/cancellations)
df = df[df['Quantity'] > 0]
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]  # optional extra filter

# 4. Remove rows with zero or negative UnitPrice (invalid)
df = df[df['UnitPrice'] > 0]

# 5. Create useful derived columns
df['TotalPrice']    = df['Quantity'] * df['UnitPrice']
df['YearMonth']     = df['InvoiceDate'].dt.to_period('M')          # for time-based analysis
df['Hour']          = df['InvoiceDate'].dt.hour
df['DayOfWeek']     = df['InvoiceDate'].dt.day_name()

# 6. Handle outliers (simple cap at 99th percentile for Quantity & UnitPrice)
q99_qty   = df['Quantity'].quantile(0.99)
q99_price = df['UnitPrice'].quantile(0.99)
df = df[(df['Quantity'] <= q99_qty) & (df['UnitPrice'] <= q99_price)]

# Final shape after cleaning
print("\nShape after cleaning:", df.shape)
print("Missing values after cleaning:\n", df.isnull().sum())
print("\nSample after cleaning:")
print(df.head())

print("\nPreprocessing completed!")

Original shape: (541909, 8)
Missing values per column:
 InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Shape after cleaning: (390260, 12)
Missing values after cleaning:
 InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
TotalPrice     0
YearMonth      0
Hour           0
DayOfWeek      0
dtype: int64

Sample after cleaning:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          Inv